In [2]:
# Vector DB인 Milvus와 관련된 모듈들은 모듈의 용도를 명확히 구분하기 위해 Vector DB 구축 단계에서 불러왔으며 해당 부분에서 코드를 확인하실 수 있습니다.
import json
import os
import subprocess
from langchain_community.document_loaders import UnstructuredHTMLLoader
from pathlib import Path
import base64
import http.client
from tqdm import tqdm
import requests

# Raw Data -> Connecting

### LangChain 활용 HTML 로딩

In [44]:
html_files_dir = Path('/Users/geesuee/jinjihan/bside_jupyter/PROGRAME/POTENDAY/2409_FAQ 응답 챗봇/2_RAG/02_potenday/potendayguide')
 
html_files = list(html_files_dir.glob("*.html"))
 
potendaydatas = []
 
for html_file in html_files:
    loader = UnstructuredHTMLLoader(str(html_file))
    document_data = loader.load()
    potendaydatas.append(document_data)
    print(f"Processed {html_file}")

Processed /Users/geesuee/jinjihan/bside_jupyter/PROGRAME/POTENDAY/2409_FAQ 응답 챗봇/2_RAG/02_potenday/potendayguide/NAVER Cloud Platform 가이드 e155fa2498a64cb39013831f05732949.html
Processed /Users/geesuee/jinjihan/bside_jupyter/PROGRAME/POTENDAY/2409_FAQ 응답 챗봇/2_RAG/02_potenday/potendayguide/포텐데이 산출물 f6471747a4a04facbaa30752bc3bbb81.html
Processed /Users/geesuee/jinjihan/bside_jupyter/PROGRAME/POTENDAY/2409_FAQ 응답 챗봇/2_RAG/02_potenday/potendayguide/🤖 [체크인] 박수많이 6👏 (낮 12시, 1회) 3e2ff65a4dc54459a1fbf859da38b07a.html
Processed /Users/geesuee/jinjihan/bside_jupyter/PROGRAME/POTENDAY/2409_FAQ 응답 챗봇/2_RAG/02_potenday/potendayguide/2단계 크레딧 5c0bf250f5c64ba1ba0d1ac74b89355e.html
Processed /Users/geesuee/jinjihan/bside_jupyter/PROGRAME/POTENDAY/2409_FAQ 응답 챗봇/2_RAG/02_potenday/potendayguide/🤖 [체크인] 박수많이 1👏 (낮 12시, 1회) abc1c213f929453ea7d661ebd3738b28.html
Processed /Users/geesuee/jinjihan/bside_jupyter/PROGRAME/POTENDAY/2409_FAQ 응답 챗봇/2_RAG/02_potenday

In [45]:
# 이중 리스트를 풀어서 하나의 리스트로 만드는 작업
potendaydatas_flattened = [item for sublist in potendaydatas for item in sublist]

# Chunking
- 문단 나누기

In [46]:
class SegmentationExecutor:
    def __init__(self, host, api_key, api_key_primary_val, request_id):
        self._host = host
        self._api_key = api_key
        self._api_key_primary_val = api_key_primary_val
        self._request_id = request_id

    def _send_request(self, completion_request):
        headers = {
            'Content-Type': 'application/json; charset=utf-8',
            'X-NCP-CLOVASTUDIO-API-KEY': self._api_key,
            'X-NCP-APIGW-API-KEY': self._api_key_primary_val,
            'X-NCP-CLOVASTUDIO-REQUEST-ID': self._request_id
        }

        conn = http.client.HTTPSConnection(self._host)
        conn.request('POST', '/testapp/v1/api-tools/segmentation/73d7ecf6f64e4986aa1ea1b01359e726', json.dumps(completion_request), headers)
        response = conn.getresponse()
        result = json.loads(response.read().decode(encoding='utf-8'))
        conn.close()
        return result

    def execute(self, completion_request):
        res = self._send_request(completion_request)
        if res['status']['code'] == '20000':
            return res['result']['topicSeg']
        else:
            return 'Error'


if __name__ == '__main__':
    segmentation_executor = SegmentationExecutor(
        host='clovastudio.apigw.ntruss.com',
        api_key='NTA0MjU2MWZlZTcxNDJiY4qFDZAWlqFMzImmP0HcHEB5yfXVQ1u0DGKUuK5/Mz8w',
        api_key_primary_val = 'OnBeBKQrOFqEElE96ackKhkzUfwOMkNbqV7bZU04',
        request_id='f560ec50-1633-46d5-a8a2-9b1f1dbada5b'
    )

    chunked_html = []
  
    for htmldata in tqdm(potendaydatas_flattened):
        try:
            request_data = {
                "postProcessMaxSize": 100,
                "alpha": -100,
                "segCnt": -1,
                "postProcessMinSize": -1,
                "text": htmldata.page_content,
                "postProcess": True
            }
             
            request_json_string = json.dumps(request_data)
            request_data = json.loads(request_json_string, strict=False)
            response_data = segmentation_executor.execute(request_data)
            result_data = [' '.join(segment) for segment in response_data]
 
        except json.JSONDecodeError as e:
            print(f"JSON decoding failed: {e}")
        except Exception as e:
            print(f"An error occurred: {e}")
         
        for paragraph in result_data:
            chunked_document = {
                "source": htmldata.metadata["source"],
                "text": paragraph
            }
            chunked_html.append(chunked_document)
  
print(len(chunked_html))

100%|██████████| 76/76 [00:14<00:00,  5.08it/s]

212


# Embedding
- 벡터화

In [47]:
class EmbeddingExecutor:
    def __init__(self, host, api_key, api_key_primary_val, request_id):
        self._host = host
        self._api_key = api_key
        self._api_key_primary_val = api_key_primary_val
        self._request_id = request_id

    def _send_request(self, completion_request):
        headers = {
            'Content-Type': 'application/json; charset=utf-8',
            'X-NCP-CLOVASTUDIO-API-KEY': self._api_key,
            'X-NCP-APIGW-API-KEY': self._api_key_primary_val,
            'X-NCP-CLOVASTUDIO-REQUEST-ID': self._request_id
        }

        conn = http.client.HTTPSConnection(self._host)
        conn.request(
            'POST', 
            '/testapp/v1/api-tools/embedding/v2/2296ea6a879a4a4498337e5f2d0cc789', 
            json.dumps(completion_request), 
            headers
        )
        response = conn.getresponse()
        result = json.loads(response.read().decode(encoding='utf-8'))
        conn.close()
        return result

    def execute(self, completion_request):
        res = self._send_request(completion_request)
        if res['status']['code'] == '20000':
            return res['result']['embedding']
        else:
            error_code = res["status"]["code"]
            error_message = res.get("status", {}).get("message", "Unknown error")
            raise ValueError(f"오류 발생: {error_code}: {error_message}")


if __name__ == '__main__':
    embedding_executor = EmbeddingExecutor(
        host='clovastudio.apigw.ntruss.com',
        api_key='NTA0MjU2MWZlZTcxNDJiY4qFDZAWlqFMzImmP0HcHEB5yfXVQ1u0DGKUuK5/Mz8w',
        api_key_primary_val = 'OnBeBKQrOFqEElE96ackKhkzUfwOMkNbqV7bZU04',
        request_id='dc14a55b-9ac5-4c8e-88cb-afb1faf1a05d'
    )

    for i, chunked_document in enumerate(tqdm(chunked_html)):
        try:
            request_json = {
                "text": chunked_document['text']
            }
            request_json_string = json.dumps(request_json)
            request_data = json.loads(request_json_string, strict=False)
            response_data = embedding_executor.execute(request_data)
        except ValueError as e:
            print(f"Embedding API Error. {e}")
        except Exception as e:
            print(f"Unexpected error: {e}")
         
        chunked_document["embedding"] = response_data


 30%|███       | 64/212 [00:15<00:21,  6.81it/s]

Embedding API Error. 오류 발생: 42901: Too many requests - rate exceeded
Embedding API Error. 오류 발생: 42901: Too many requests - rate exceeded
Embedding API Error. 오류 발생: 42901: Too many requests - rate exceeded
Embedding API Error. 오류 발생: 42901: Too many requests - rate exceeded


 32%|███▏      | 68/212 [00:15<00:14, 10.06it/s]

Embedding API Error. 오류 발생: 42901: Too many requests - rate exceeded
Embedding API Error. 오류 발생: 42901: Too many requests - rate exceeded
Embedding API Error. 오류 발생: 42901: Too many requests - rate exceeded
Embedding API Error. 오류 발생: 42901: Too many requests - rate exceeded


 34%|███▍      | 72/212 [00:15<00:11, 12.39it/s]

...

Embedding API Error. 오류 발생: 42901: Too many requests - rate exceeded
Embedding API Error. 오류 발생: 42901: Too many requests - rate exceeded
Embedding API Error. 오류 발생: 42901: Too many requests - rate exceeded
Embedding API Error. 오류 발생: 42901: Too many requests - rate exceeded


 99%|█████████▉| 210/212 [00:25<00:00, 16.74it/s]

Embedding API Error. 오류 발생: 42901: Too many requests - rate exceeded
Embedding API Error. 오류 발생: 42901: Too many requests - rate exceeded
Embedding API Error. 오류 발생: 42901: Too many requests - rate exceeded
Embedding API Error. 오류 발생: 42901: Too many requests - rate exceeded


100%|██████████| 212/212 [00:25<00:00,  8.42it/s]

Embedding API Error. 오류 발생: 42901: Too many requests - rate exceeded
Embedding API Error. 오류 발생: 42901: Too many requests - rate exceeded


In [48]:
dimension_set = set()
 
for item in chunked_html:
    if "embedding" in item:
        dimension = len(item["embedding"])
        dimension_set.add(dimension)
 
print("임베딩된 벡터들의 차원:", dimension_set)

임베딩된 벡터들의 차원: {1024}


In [50]:
chunked_html[100]

{'source': '/Users/geesuee/jinjihan/bside_jupyter/PROGRAME/POTENDAY/2409_FAQ 응답 챗봇/2_RAG/02_potenday/potendayguide/비사이드 X 네이버 클라우드 플랫폼 e884b04cad38442f91eb1e70aee6c868.html',
 'text': 'NAVER Cloud Platform 사용 주의 사항 NAVER Cloud Platform 관련 FAQ',
 'embedding': [0.47265625,
  0.31762695,
  -0.81103516,
  1.1787109,
  -0.85253906,
  -1.1875,
  1.1962891,
  -1.7382812,
  -0.09289551,
  0.6064453,
  1.0634766,
  0.3395996,
  0.52246094,
  0.107055664,
  -0.15014648,
  -0.9760742,
  0.035064697,
  -1.8496094,
  0.42358398,
  0.35205078,
  -0.38378906,
  -0.26513672,
  0.4543457,
  -0.2668457,
  -0.3334961,
  -0.76708984,
  0.17138672,
  1.0615234,
  0.7495117,
  1.2158203,
  0.41601562,
  -0.47338867,
  0.5625,
  -1.2529297,
  0.5800781,
  -0.6899414,
  0.39038086,
  -0.41015625,
  -0.58740234,
  1.3076172,
  -0.8466797,
  0.5683594,
  -0.29492188,
  -0.51171875,
  0.6669922,
  -0.5048828,
  0.17456055,
  0.33764648,
  -0.7597656,
  -0.091430664,
  0.06915283,
  0.61865234,


# Vector DB
- Docker 로 벡터 DB Milvus 설치, 실행한 뒤 변환한 벡터 저장

### Collections & ID 생성

In [51]:
from pymilvus import connections, FieldSchema, CollectionSchema, DataType, Collection, utility

In [52]:
# Milvus 서버에 연결 (호스트는 localhost, 포트는 19530)
connections.connect("default", host="localhost", port="19530")
# 연결 성공 여부 확인
print(connections.get_connection_addr("default"))

{'address': 'localhost:19530', 'user': ''}


In [53]:
# 기존 컬렉션 드롭

# Milvus 서버 연결
connections.connect("default", host="localhost", port="19530")

# 드롭할 컬렉션 이름
collection_name = "potenday"

# 컬렉션을 가져옴
collection = Collection(name=collection_name)

# 컬렉션 드롭 (모든 데이터 삭제)
collection.drop()

print(f"컬렉션 '{collection_name}'이 삭제되었습니다.")

컬렉션 'potenday'이 삭제되었습니다.


In [54]:
connections.connect("default", host="localhost", port="19530")
  
fields = [
    FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=True),
    FieldSchema(name="source", dtype=DataType.VARCHAR, max_length=3000),
    FieldSchema(name="text", dtype=DataType.VARCHAR, max_length=9000),
    FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=1024)
]

schema = CollectionSchema(fields, description="포텐데이 FAQ 테스트")

collection_name = "potenday"
collection = Collection(name=collection_name, schema=schema, using='default', shards_num=2)

for item in chunked_html:
    source_list = [item['source']]
    text_list = [item['text']]
    embedding_list = [item['embedding']]
    
    entities = [
        source_list,
        text_list,
        embedding_list
    ]
    
    insert_result = collection.insert(entities)
    print("데이터 Insertion이 완료된 ID:", insert_result.primary_keys)

print("데이터 Insertion이 전부 완료되었습니다")

데이터 Insertion이 완료된 ID: [452338135758013232]
데이터 Insertion이 완료된 ID: [452338135758013234]
데이터 Insertion이 완료된 ID: [452338135758013236]
데이터 Insertion이 완료된 ID: [452338135758013238]
데이터 Insertion이 완료된 ID: [452338135758013240]
데이터 Insertion이 완료된 ID: [452338135758013242]
데이터 Insertion이 완료된 ID: [452338135758013244]
데이터 Insertion이 완료된 ID: [452338135758013246]
데이터 Insertion이 완료된 ID: [452338135758013248]
데이터 Insertion이 완료된 ID: [452338135758013250]
데이터 Insertion이 완료된 ID: [452338135758013252]
데이터 Insertion이 완료된 ID: [452338135758013254]
데이터 Insertion이 완료된 ID: [452338135758013256]
데이터 Insertion이 완료된 ID: [452338135758013258]
데이터 Insertion이 완료된 ID: [452338135758013260]
데이터 Insertion이 완료된 ID: [452338135758013262]
데이터 Insertion이 완료된 ID: [452338135758013264]
데이터 Insertion이 완료된 ID: [452338135758013266]
데이터 Insertion이 완료된 ID: [452338135758013268]
데이터 Insertion이 완료된 ID: [452338135758013270]
데이터 Insertion이 완료된 ID: [452338135758013272]
데이터 Insertion이 완료된 ID: [452338135758013274]
데이터 Insertion이 완료된 ID: [45233813

### Indexing

In [55]:
index_params = {
    "metric_type": "IP",
    "index_type": "HNSW",
    "params": {
        "M": 8,
        "efConstruction": 200
    }
}
 
collection = Collection("potenday")
collection.create_index(field_name="embedding", index_params=index_params)
utility.index_building_progress("potenday")
 
print([index.params for index in collection.indexes])

[{'metric_type': 'IP', 'index_type': 'HNSW', 'params': {'M': 8, 'efConstruction': 200}}]


### 재실행 시 Milvus 로딩

In [65]:
# 재실행 시 로딩
connections.connect("default", host="localhost", port="19530")

# 불러올 collection 이름을 넣는 곳
collection = Collection("potenday")
collection.create_index(field_name="embedding", index_params=index_params)
utility.index_building_progress("potenday")
 
print([index.params for index in collection.indexes])

<LoadState: NotLoad>

# Retrieval -> HyperCLOVA X

In [58]:
# -*- coding: utf-8 -*-

import requests


class CompletionExecutor:
    def __init__(self, host, api_key, api_key_primary_val, request_id):
        self._host = host
        self._api_key = api_key
        self._api_key_primary_val = api_key_primary_val
        self._request_id = request_id

    def execute(self, completion_request, response_type="stream"):
        headers = {
            'X-NCP-CLOVASTUDIO-API-KEY': self._api_key,
            'X-NCP-APIGW-API-KEY': self._api_key_primary_val,
            'X-NCP-CLOVASTUDIO-REQUEST-ID': self._request_id,
            'Content-Type': 'application/json; charset=utf-8',
            'Accept': 'text/event-stream'
        }

        final_answer = ""

        with requests.post(
            self._host + '/testapp/v1/chat-completions/HCX-DASH-001',
            headers=headers, 
            json=completion_request, 
            stream=True
        ) as r:
            if response_type == "stream":
                longest_line = ""
                for line in r.iter_lines():
                    if line:
                        decoded_line = line.decode("utf-8")
                        if decoded_line.startswith("data:"):
                            event_data = json.loads(decoded_line[len("data:"):])
                            message_content = event_data.get("message", {}).get("content", "")
                            if len(message_content) > len(longest_line):
                                longest_line = message_content
                final_answer = longest_line
            elif response_type == "single":
                final_answer = r.json()  # 가정: 단일 응답이 JSON 형태로 반환됨
        
        return final_answer

In [56]:
# 사용자의 쿼리를 임베딩하는 함수를 먼저 정의
def query_embed(text: str):
    request_data = {"text": text}
    response_data = embedding_executor.execute(request_data)
    return response_data

In [65]:
def html_chat(realquery: str) -> str:

    query_vector = query_embed(realquery)
 
    collection.load()
 
    search_params = {"metric_type": "IP", "params": {"ef": 64}}
    results = collection.search(
        data=[query_vector],  # 검색할 벡터 데이터
        anns_field="embedding",  # 검색을 수행할 벡터 필드 지정
        param=search_params,
        limit=10,
        output_fields=["source", "text"]
    )
 
    reference = []
 
    for hit in results[0]:
        distance = hit.distance
        source = hit.entity.get("source")
        text = hit.entity.get("text")
        reference.append({"distance": distance, "source": source, "text": text})
 
    completion_executor = CompletionExecutor(
        host='https://clovastudio.stream.ntruss.com',
        api_key='NTA0MjU2MWZlZTcxNDJiY4qFDZAWlqFMzImmP0HcHEB5yfXVQ1u0DGKUuK5/Mz8w',
        api_key_primary_val='OnBeBKQrOFqEElE96ackKhkzUfwOMkNbqV7bZU04',
        request_id='01ed5897-a7af-49b2-81d5-05b86b73254d'
    )

    preset_texts = [
        {"role":"system","content":"- 너의 역할은 사용자의 질문에 reference를 바탕으로 답변하는거야.\n- 상냥하고 친절하고 귀여운 어투로 대답해줘.\n- 너가 가지고 있는 지식은 모두 배제하고, 주어진 reference의 내용만을 바탕으로 답변해야해.\n- 답변의 출처가 되는 'source'도 답변과 함께 {출처:}의 형태로 제공해야해.\n- 만약 사용자의 질문이 reference와 관련이 없다면, {오잉님, 도와주세요.}라고만 반드시 말해야해.\n- 네가 가진 지식은 반드시 다 배제하고 주어진 reference에 있는 내용만을 바탕으로 대답해."},
    ]
 
    for ref in reference:
        preset_texts.append(
            {
                "role": "system",
                "content": f"reference: {ref['text']}, url: {ref['source']}"
            }
        )
 
    preset_texts.append({"role": "user", "content": realquery})
 
    request_data = {
        "messages": preset_texts,
        "topP": 0.6,
        "topK": 0,
        "maxTokens": 1024,
        "temperature": 0.5,
        "repeatPenalty": 1.2,
        "stopBefore": [],
        "includeAiFilters": False
    }
 
    # LLM 생성 답변 반환
    response_data = completion_executor.execute(request_data)
 
    return response_data

In [61]:
html_chat("포텐데이 중간 산출물은 언제 내나요?")

'중간 산출물은 DAY 5에 제출합니다.  \n\n포텐데이 슬랙 > 포텐데이 채널에 직접 소개 포텐데이 체크인 > /프로젝트 산출물 호출해서 서비스 소개 등록 중간 산출물은 밤 12시 마감 됩니다. Day 5 이후 제출도 가능합니다. (하지만, 패널티 있어요.)\n\n자세한 내용은 포텐데이 산출물 가이드를 참고하세요.\n\n- 포텐데이 산출물 : /Users/geesuee/jinjihan/bside_jupyter/PROGRAME/POTENDAY/2409_FAQ 응답 챗봇/2_RAG/02_potenday/potendayguide/포텐데이 산출물 f6471747a4a04facbaa30752bc3bbb81.html'

In [36]:
html_chat("팀을 만들었는데 중간에 그만두고 싶으면 어떻게 하나요?")

'팀을 만들었는데 중간에 그만두고 싶다면 포텐데이 체크인 > 사이드카 > 프로젝트 하차 신청을 해 주세요. 운영자 확인 후 하차 프로세스가 진행되니 슬랙 DM(포텐데이 체크인)을 꼼꼼히 확인하세요.\n\n- 출처 : 프로젝트 하차 신청은 어떻게 하나요? 기간별 진행중 유형 프로젝트 중단⛔ 프로그램 시작 전후, 프로젝트 하차하고 싶다면 포텐데이 체크인 > 사이드카 > 프로젝트 하차 신청해 주세요. 운영자 확인 후 하차 프로세스가 진행되니 슬랙 DM(포텐데이 체크인)을 꼼꼼히 확인하세요., url : /Users/geesuee/jinjihan/bside_jupyter/PROGRAME/POTENDAY/2409_FAQ 응답 챗봇/2_RAG/02_potenday/potendayguide/프로젝트 하차 신청은 어떻게 하나요 25a821809506497ab81c241b43f4f082.html'

In [38]:
html_chat("팀빌딩이 안되면 다크호스가 되는건가요? 다크호스가 뭔가요?")

'네. 팀 빌딩이 안 되면 다크호스로 다른 팀에 합류할 수 있어요.\n\n팀이 확정되지 않은 멤버 (=다크호스)와 영입을 희망하는 팀이 만나 최종 팀 빌딩을 합니다. {출처:/Users/geesuee/jinjihan/bside_jupyter/PROGRAME/POTENDAY/2409_FAQ 응답 챗봇/2_RAG/02_potenday/potendayguide/[팀 빌딩] 다크호스 합류 2e437c1d866e4ef2869cd1eec6ae4b7f.html}'

In [40]:
html_chat("사이드카 어떻게 호출하나요?")

'포텐데이 슬랙 아무 채널 메시지 입력창에 /사이드카 명령어 입력 DM 말고 사이드카🚨 발동하세요.\n- 출처 : /Users/geesuee/jinjihan/bside_jupyter/PROGRAME/POTENDAY/2409_FAQ 응답 챗봇/2_RAG/02_potenday/potendayguide/포텐데이 체크인 71ba85b21a0747d99353e7f3f3e60ca7.html'

In [62]:
html_chat("입장 체크인을 못 받았어요 어떡하죠?")

'포텐데이 슬랙 입장 체크인을 받지 못하셨다면, 잠시 후에 다시 시도해 보세요.  \n\n만약 계속해서 체크인을 받지 못하신다면, 슬랙 채널 메뉴 하단의 포텐데이 체크인 DM을 확인해 주세요.'

In [66]:
html_chat("고양이가 좋아하는 음식은?")

'고양이가 좋아하는 음식에 대해 궁금하시군요. 고양이가 좋아하는 음식은 다양하지만 일반적으로 선호하는 음식은 다음과 같습니다.  \n\n- 고양이가 좋아하는 음식 : 생선, 고기, 계란, 우유, 통조림 등입니다.\n\n위의 음식들은 고양이의 건강에 도움이 될 수 있지만, 반드시 고양이의 건강 상태와 체질을 고려하여 적절한 양과 종류를 선택해야 합니다.'